In [2]:
import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

In [3]:
txt='''inp w
mul x 0
add x z
mod x 26
div z 1
add x 14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 8
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 15
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 13
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 2
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -10
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 1
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -3
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 5
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 10
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 12
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 6
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 1
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 12
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -6
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 9
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -6
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 14
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -2
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -9
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 2
mul y x
add z y'''

In [4]:
program_str=[x.split(' ') for x in txt.split('\n')]

In [5]:
i=2

r=program_str[18*i:18*(i+1)]
[[int(program_str[18*i:18*(i+1)] [c][2]) for c in [4,5,15]] for i in range(14)]

[[1, 14, 8],
 [1, 15, 11],
 [1, 13, 2],
 [26, -10, 11],
 [1, 14, 1],
 [26, -3, 5],
 [26, -14, 10],
 [1, 12, 6],
 [1, 14, 1],
 [1, 12, 11],
 [26, -6, 9],
 [26, -6, 14],
 [26, -2, 11],
 [26, -9, 2]]

In [32]:
def simplificar_lut(lut):
    for k in lut.keys():
        if len(lut[k])>=9:
            lut[k]=simplificar_input_value(lut[k])
        if len(lut[k])>1:
            winner=np.argmax([str(w) for w in lut[k]])
            lut[k]=[lut[k][winner]]
            
def simplificar_input_value (input_values_in):
    input_values=input_values_in.copy()    
    _,unique_index=np.unique([str(x) for x in input_values],return_index=True)
    input_values=np.array(input_values)[unique_index]
    non_zero_indexes=np.where(input_values.max(axis=0)>0)[0]
    for eliminar in non_zero_indexes:
        masked_input_values=input_values.copy()
        masked_input_values[:,eliminar]=0
        _,indexes,counts=np.unique([str(x) for x in masked_input_values],return_index=True,return_counts=True)
        to_remove=masked_input_values[indexes[counts==9],:]
        remove_rows=None
        for remove in to_remove:
            if remove_rows is None:
                remove_rows=((masked_input_values==remove).min(axis=1))
            else:
                remove_rows=remove_rows | ((masked_input_values==remove).min(axis=1))
        if remove_rows is not None:
            if sum(remove_rows)==masked_input_values.shape[0]:
                input_values=to_remove
            else:
                input_values=np.concatenate([input_values[~remove_rows,:],to_remove],axis=0)
    
    #return list(input_values)
    
    return list(input_values)
    
    #if input_values.shape[0]>1:
    #    winner=np.argmax([str(input_values[i,:] for i in range(input_values.shape[0]))])
    #    return list(input_values[winner,:])
    #else:
    #    return list(input_values)

In [43]:
np.floor(0.5)

0.0

In [44]:
def inp_lut(index):
    return {i:[np.array([i if index==d else 0 for d in range(14)])] for i in range(1,10)}

def combina_entradas(v1,v2):
    difs=v1!=v2
    valid_difs=((v1==0)|(v2==0))
    if sum(difs*(1-(valid_difs))):
        c=None
    else:
        c=v1+(v2*(v1==0))
    return c
    
def cns_lut(value):
    return {value:[np.array([0 for d in range(14)])]}

def simplify_lut(lut):
    return lut

def mul_lut(lut1,lut2):
    if type(lut2)==int:
        ret_val= mul_lut(lut1,cns_lut(lut2))
    else:
        ret_val={}
        for k1 in lut1.keys():
            for k2 in lut2.keys():
                new_sequence=[]
                for v1 in lut1[k1]:
                    for v2 in lut2[k2]:
                        c=combina_entradas(v1,v2)
                        if c is not None:
                            new_sequence.append(c.copy())
                if len(new_sequence):
                    if k1*k2 not in ret_val.keys():
                        ret_val[k1*k2]=new_sequence.copy()
                    else:
                        ret_val[k1*k2]+=new_sequence.copy()
                    
    return ret_val

def div_lut(lut1,lut2):
    if type(lut2)==int:
        ret_val= div_lut(lut1,cns_lut(lut2))
    else:
        ret_val={}
        for k1 in lut1.keys():
            for k2 in lut2.keys():
                out_val=int(np.floor(k1/k2))
                
                
                new_sequence=[]
                for v1 in lut1[k1]:
                    for v2 in lut2[k2]:
                        c=combina_entradas(v1,v2)
                        if c is not None:
                            new_sequence.append(c.copy())
                if len(new_sequence):
                    if out_val not in ret_val.keys():
                        ret_val[out_val]=new_sequence.copy()
                    else:
                        ret_val[out_val]+=new_sequence.copy()

                
                
    return ret_val
def mod_lut(lut1,lut2):
    if type(lut2)==int:
        ret_val= mod_lut(lut1,cns_lut(lut2))
    else:
        ret_val={}
        for k1 in lut1.keys():
            for k2 in lut2.keys():
                out_val=int(round(k1%k2))
                new_sequence=[]
                for v1 in lut1[k1]:
                    for v2 in lut2[k2]:
                        c=combina_entradas(v1,v2)
                        if c is not None:
                            new_sequence.append(c.copy())
                if len(new_sequence):
                    if out_val not in ret_val.keys():
                        ret_val[out_val]=new_sequence.copy()
                    else:
                        ret_val[out_val]+=new_sequence.copy()
                
    return ret_val

def eql_lut(lut1,lut2):
    if type(lut2)==int:
        ret_val= eql_lut(lut1,cns_lut(lut2))
    else:
        ret_val={}
        for k1 in lut1.keys():
            for k2 in lut2.keys():
                out_val=int(k1==k2)
                new_sequence=[]
                for v1 in lut1[k1]:
                    for v2 in lut2[k2]:
                        c=combina_entradas(v1,v2)
                        if c is not None:
                            new_sequence.append(c.copy())
                if len(new_sequence):
                    if out_val not in ret_val.keys():
                        ret_val[out_val]=new_sequence.copy()
                    else:
                        ret_val[out_val]+=new_sequence.copy()

                            
        
    return ret_val

def add_lut(lut1,lut2):
    if type(lut2)==int:
        ret_val= add_lut(lut1,cns_lut(lut2))
    else:
        ret_val={}
        for k1 in lut1.keys():
            for k2 in lut2.keys():
                out_val=int(k1+k2)
                new_sequence=[]
                for v1 in lut1[k1]:
                    for v2 in lut2[k2]:
                        c=combina_entradas(v1,v2)
                        if c is not None:
                            new_sequence.append(c.copy())
                if len(new_sequence):
                    if out_val not in ret_val.keys():
                        ret_val[out_val]=new_sequence.copy()
                    else:
                        ret_val[out_val]+=new_sequence.copy()

                            
        
    return ret_val

In [45]:
operator_signs={'inp':'d','mul':'*','add':'+','div':'/','mod':'%','eql':'=='}
class ALULUT():
    
    def __init__(self):
        self.xyzw=[cns_lut(0),cns_lut(0),cns_lut(0),cns_lut(0)]
        self.n_input=0
        
    def processar_line_code (self,command_str):
        register=list('xyzw').index(command_str[1])
        
        
        if command_str[0]=='inp':
            self.xyzw[register]=inp_lut(self.n_input)
            self.n_input+=1
        else:
            if command_str[2] in 'xyzw': ## operacion entre registros
                register2=list('xyzw').index(command_str[2])
                if command_str[0]=='add':
                    self.xyzw[register]=add_lut(self.xyzw[register],self.xyzw[register2])
                elif command_str[0]=='mul':
                    self.xyzw[register]=mul_lut(self.xyzw[register],self.xyzw[register2])
                elif command_str[0]=='mod':
                    self.xyzw[register]=mod_lut(self.xyzw[register],self.xyzw[register2])
                elif command_str[0]=='div':
                    self.xyzw[register]=div_lut(self.xyzw[register],self.xyzw[register2])
                elif command_str[0]=='eql':
                    self.xyzw[register]=eql_lut(self.xyzw[register],self.xyzw[register2])
            else:
                operator=int(command_str[2]) ## operacion con constante
                if command_str[0]=='add':
                    self.xyzw[register]=add_lut(self.xyzw[register],operator)
                elif command_str[0]=='mul':
                    self.xyzw[register]=mul_lut(self.xyzw[register],operator)
                elif command_str[0]=='mod':
                    self.xyzw[register]=mod_lut(self.xyzw[register],operator)
                elif command_str[0]=='div':
                    self.xyzw[register]=div_lut(self.xyzw[register],operator)
                elif command_str[0]=='eql':
                    self.xyzw[register]=eql_lut(self.xyzw[register],operator)
            simplificar_lut(self.xyzw[register])
                
                
    @staticmethod
    def decode_text(txt):
        return [x.split(' ') for x in txt.split('\n')]
    
    def processar_programa(self,programa):
        
        for linea in tqdm(programa):
            
            self.processar_line_code(linea)
            
    def processar_programa_txt(self,txt):
        
        programa=self.decode_text(txt)
        for linea in programa:
            
            self.processar_line_code(linea)

                    
                
            
        

In [46]:
mins=[]
zs=[]
for i in range(14):
    print(f'------------------------\niteration : {i}')
    my_ALU=ALULUT()
    my_ALU.processar_programa(program_str[:(i+1)*18])
    #print(program_str[:(i+1)*18])
    my_ALU.xyzw[2].keys()
    zs.append(deepcopy(my_ALU.xyzw[2]))
    
    #print(f'mod 26: {np.unique([(x)%26 for x in my_ALU.xyzw[2].keys()])}')
    #print(f'//26%26 {np.unique([(x//26)%26 for x in my_ALU.xyzw[2].keys()])}')
    #print(f'//26%26 {np.unique([(x//26//26)%26 for x in my_ALU.xyzw[2].keys()])}')
    #mins.append(min(my_ALU.xyzw[2].keys()))
    

 81%|████████  | 58/72 [00:00<00:00, 467.03it/s]

------------------------
iteration : 0
------------------------
iteration : 1
------------------------
iteration : 2
------------------------
iteration : 3


 64%|██████▍   | 58/90 [00:00<00:00, 461.19it/s]

------------------------
iteration : 4


 54%|█████▎    | 58/108 [00:00<00:00, 459.10it/s]

...

 25%|██▍       | 58/234 [00:00<00:00, 496.82it/s]

------------------------
iteration : 12


 23%|██▎       | 58/252 [00:00<00:00, 477.06it/s]

------------------------
iteration : 13


100%|██████████| 252/252 [00:14<00:00, 17.17it/s] 


In [47]:
zs[-1]

{162979: [array([1, 9, 9, 1, 9, 7, 6, 1, 1, 9, 9, 0, 9, 9])],
 163005: [array([1, 9, 9, 1, 9, 7, 6, 1, 2, 9, 9, 0, 9, 9])],
 163031: [array([1, 9, 9, 1, 9, 7, 6, 1, 3, 9, 9, 0, 9, 9])],
 163057: [array([1, 9, 9, 1, 9, 7, 6, 1, 4, 9, 9, 0, 9, 9])],
 163083: [array([1, 9, 9, 1, 9, 7, 6, 1, 5, 9, 9, 0, 9, 9])],
 163109: [array([1, 9, 9, 1, 9, 7, 6, 1, 6, 9, 9, 0, 9, 9])],
 163135: [array([1, 9, 9, 1, 9, 7, 6, 1, 7, 9, 9, 0, 9, 9])],
 163161: [array([1, 9, 9, 1, 9, 7, 6, 1, 8, 9, 9, 0, 9, 9])],
 163187: [array([1, 9, 9, 1, 9, 7, 6, 1, 9, 9, 9, 0, 9, 9])],
 163655: [array([1, 9, 9, 1, 9, 7, 6, 2, 1, 9, 9, 0, 9, 9])],
 163681: [array([1, 9, 9, 1, 9, 7, 6, 2, 2, 9, 9, 0, 9, 9])],
 163707: [array([1, 9, 9, 1, 9, 7, 6, 2, 3, 9, 9, 0, 9, 9])],
 163733: [array([1, 9, 9, 1, 9, 7, 6, 2, 4, 9, 9, 0, 9, 9])],
 163759: [array([1, 9, 9, 1, 9, 7, 6, 2, 5, 9, 9, 0, 9, 9])],
 163785: [array([1, 9, 9, 1, 9, 7, 6, 2, 6, 9, 9, 0, 9, 9])],
 163811: [array([1, 9, 9, 1, 9, 7, 6, 2, 7, 9, 9, 0, 9, 9])],
 163837:

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,14))


i=7
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'*k',alpha=0.15)

i=6
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'*b',alpha=0.15)


i=5
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'oy',alpha=0.15)


i=4
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'ok',alpha=0.15)


i=3
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'og',alpha=0.15)

i=2
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'ob',alpha=0.15)
i=1
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'or',alpha=0.15)

i=0
plt.plot([int(round(x/26)) for x in  zs[i].keys()],[round(x%26) for x in  zs[i].keys()],'om',alpha=0.15)

x,y=get_track('1111111')
plt.plot(y[1:],x[1:])

x,y=get_track('12345644444')
plt.plot(y[1:],x[1:],'r')


plt.show()

In [ ]:
s='11111'

In [ ]:
def get_track(s):
    xf=[]
    yf=[]
    for i in range(6):
        print(i)
        ls=str([int(x) for x in list(s[:i+1])])
        r=[k for k in zs[i].keys() if max([str(list(y)[:i+1])==ls for y in zs[i][k]])][0]
        xf.append(r%26)
        yf.append(int(round(r/26)))
    return xf,yf
        
    

In [ ]:
zs[-1]

In [ ]:
s='222222222'
ls=str([int(x) for x in list(s[:i+1])])
ls

In [ ]:
[k for k in zs[i].keys() if max([str(list(y)[:4])=='[2, 2, 2, 2]' for y in zs[i][k]])]

In [ ]:
([len(x) for x in zs])


In [ ]:
zs[i][7102]